# QUANTUM SUPPORT VECTOR MACHINES

In [ ]:
pip install pennylane==0.26

In [ ]:
import numpy as np

seed = 1234
np.random.seed(seed)

In [ ]:
from sklearn.datasets import load_wine

x,y = load_wine(return_X_y = True)

In [ ]:
x = x[:59+71]
y = y[:59+71]

In [ ]:
from sklearn.model_selection import train_test_split

x_tr, x_test, y_tr, y_test = train_test_split(x, y, train_size = 0.9)

In [ ]:
from sklearn.preprocessing import MaxAbsScaler

scaler = MaxAbsScaler()
x_tr = scaler.fit_transform(x_tr)

In [ ]:
x_test = scaler.transform(x_test)
x_test = np.clip(x_test,-1,1)

In [ ]:
import pennylane as qml

nqubits = 4
dev = qml.device("lightning.qubit", wires = nqubits)

@qml.qnode(dev)
def kernel_circ(a, b):
    qml.AmplitudeEmbedding(
        a, wires=range(nqubits), pad_with=0, normalize=True)
    qml.adjoint(qml.AmplitudeEmbedding(
        b, wires=range(nqubits), pad_with=0, normalize=True))
    return qml.probs(wires = range(nqubits))

In [ ]:
from sklearn.svm import SVC
def qkernel(A, B):
    return np.array([[kernel_circ(a, b)[0] for b in B] for a in A])

svm = SVC(kernel = qkernel).fit(x_tr, y_tr)

In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(svm.predict(x_test), y_test))